<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 371 Homework 10

### Problem 0 (3 points)

## Part 1: Mathematics of Correlation and Convolution

### Problem 1.1 (Exam Style)

### Problem 1.2 (Exam Style)

## Part 2: Coding Correlation and Convolution

In [1]:
from urllib.request import urlretrieve
from os import path as osp


def retrieve(file_name, semester='fall22', course='371', homework=10):
    if osp.exists(file_name):
        print('Using previously downloaded file {}'.format(file_name))
    else:
        fmt = 'https://www2.cs.duke.edu/courses/{}/compsci{}/homework/{}/{}'
        url = fmt.format(semester, course, homework, file_name)
        urlretrieve(url, file_name)
        print('Downloaded file {}'.format(file_name))

In [2]:
retrieve('oracle.py')

Downloaded file oracle.py


In [3]:
from oracle import oracle, show


def test(function, a=[1, 2, 3, 4], b=[5, 6]):
    for operator in ('correlate', 'convolve'):
        for mode in ('full', 'same', 'valid'):
            for f, g in ((a, b), (b, a)):
                c = function(operator, f, g, mode)
                show(operator, f, g, mode, c)

### Problem 2.1

### Problem 2.2

### Problem 2.3

## Part 3: Back-Propagation through a Convolution 

### Problem 3.1 (Exam Style)

$$
\begin{eqnarray*}
c_0 &=& a_0 b_0 \\
c_1 &=& a_0 b_1 + a_1 b_0 \\
c_2 &=& a_0 b_2 + a_1 b_1 + a_2 b_0 \\
c_3 &=& a_1 b_2 + a_2 b_1 + a_3 b_0 \\
c_4 &=& a_2 b_2 + a_3 b_1 \\
c_5 &=& a_3 b_2
\end{eqnarray*}
$$

## Part 4: Network Back-Propagation 

![tiny network](https://courses.cs.duke.edu//fall22/compsci371/homework/10/tiny_network.png)

### Problem 4.1 (Exam Style)

### Problem 4.2 (Exam Style)

$$
\begin{eqnarray*}
g_{\hat{y}} &=& \hat{y} - y = 3 - 5 = -2 \\
g_b &=& g_{\hat{y}} \cdot 1 = -2 \\
g_{v_2} &=& g_{\hat{y}}\ z_2 = -2 \cdot 7 = -14\\
g_{v_1} &=&  \\
g_{z_2} &=&  \\
g_{z_1} &=&  \\
g_{a_2} &=&  \\
g_{a_1} &=&  \\
g_{u_{23}} &=&  \\
g_{u_{22}} &=&  \\
g_{u_{21}} &=&  \\
g_{u_{13}} &=&  \\
g_{u_{12}} &=&  \\
g_{u_{11}} &=& 
\end{eqnarray*}
$$

Collecting the relevant entries in reverse order yields

$$
g_{\mathbf{w}} = ()\;.
$$

## Part 5: MNIST Digit Classification

In [12]:
import pickle


file_name = 'mnist.pickle'
retrieve(file_name)
with open(file_name, 'rb') as file:
    mnist = pickle.load(file)

Downloaded file mnist.pickle


In [13]:
def print_result(h, d):
    accuracy = {
        'train': h.score(d['train']['x'], d['train']['y']) * 100,
        'test': h.score(d['test']['x'], d['test']['y']) * 100
    }
    print('training accuracy: {:.2f} percent'.format(accuracy['train']))
    print('test accuracy: {:.2f} percent'.format(accuracy['test']))
    max_points = 20
    p = (accuracy['test'] - 90.) / (96. - 90.) * max_points
    p = min((max_points, max((0, p))))
    p = round(p)
    print('{} out of {} points'.format(p, max_points))

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.exceptions import ConvergenceWarning
import warnings


def experiment(data, hidden_layer_sizes, max_iter, alpha,
               learning_rate_init, verbose=False):
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        max_iter=max_iter,
        alpha=alpha,
        learning_rate_init=learning_rate_init,
        learning_rate='constant',
        solver='sgd',
        random_state=1,
        verbose=verbose
    )
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore", category=ConvergenceWarning, module="sklearn")
        mlp.fit(data['train']['x'], data['train']['y'])
    print_result(mlp, data)

### Problem 5.1 (Exam Style)

### Problem 5.2

In [15]:
experiment(
    mnist,
    hidden_layer_sizes=(20, 5),
    max_iter=20,
    alpha=1e-8,
    learning_rate_init=0.5,
    verbose=False
)

training accuracy: 10.56 percent
test accuracy: 10.40 percent
0 out of 20 points
